# Analyzing Cryptocurrencies with Python

[]('https://bitcoin.org/img/video/video_wuc.jpg')

![](https://bitcoin.org/img/video/video_wuc.jpg)

## Python and Cryptocurrencies

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from bitcoin_price import btc_usd_datasets, df_scatter, merge_dfs_on_column
from datetime import datetime

Loaded BCHARTS/KRAKENUSD from cache
Loaded BCHARTS/COINBASEUSD from cache
Loaded BCHARTS/BITSTAMPUSD from cache
Loaded BCHARTS/ITBITUSD from cache


In [2]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [3]:
def get_json_data(json_url, cache_path):
    '''Download and cache JSON data, return as a dataframe.'''
    try:        
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(json_url))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(json_url, cache_path))
    return df

In [6]:
# Retrieve data from Poloniex exchange
base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # get data from the start of 2015
end_date = datetime.now() # up until today
period = 86400 # pull daily data (86,400 seconds per day)

def get_crypto_data(poloniex_pair):
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), period)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df

In [16]:
# Analyze Nine Altcoins
altcoins = ['ETH','LTC','XRP','ETC','STR','DASH','ZEC','XMR','XEM']

altcoin_data = {}
for altcoin in altcoins:
    coinpair = 'BTC_{}'.format(altcoin)
    crypto_price_df = get_crypto_data(coinpair)
    altcoin_data[altcoin] = crypto_price_df

Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=1420088400.0&end=1509285055.923352&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_LTC&start=1420088400.0&end=1509285055.923352&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XRP&start=1420088400.0&end=1509285055.923352&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETC&start=1420088400.0&end=1509285055.923352&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_STR&start=1420088400.0&end=1509285055.923352&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_DASH&start=1420088400.0&end=1509285055.923352&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ZEC&start=1420088400.0&end=1509285055.923352&period=8

In [19]:
# Find the latest Ethereum data
altcoin_data['ETH'].tail()

close      high       low      open    quoteVolume  \
date                                                                
2017-10-24  0.053761  0.056368  0.048205  0.048205  197553.795798   
2017-10-25  0.051703  0.054401  0.051600  0.053761   54113.256275   
2017-10-26  0.050102  0.052360  0.049500  0.051610   52757.468619   
2017-10-27  0.051487  0.051951  0.049584  0.050091   41502.387529   
2017-10-28  0.051291  0.052100  0.050210  0.051487   33932.215283   

                  volume  weightedAverage  
date                                       
2017-10-24  10574.722384         0.053528  
2017-10-25   2881.731116         0.053254  
2017-10-26   2672.997796         0.050666  
2017-10-27   2113.216252         0.050918  
2017-10-28   1737.814762         0.051214

In [26]:
# Export the latest Ethereum pricing data to excel
altcoin_data['ETH'].to_csv('ethereum.csv')

In [9]:
# Calculate USD Price as a new column in each altcoin dataframe
for altcoin in altcoin_data.keys():
    altcoin_data[altcoin]['price_usd'] =  altcoin_data[altcoin]['weightedAverage'] * btc_usd_datasets['avg_btc_price_usd']

In [10]:
# Merge USD price of each altcoin into one dataframe 
combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'price_usd')

In [11]:
# Add BTC price to the dataframe
combined_df['BTC'] = btc_usd_datasets['avg_btc_price_usd']

In [12]:
# Graph chosen altocoin prices
df_scatter(combined_df, 'Cryptocurrency Prices (USD)', seperate_y_axis=False, y_axis_label='Coin Value (USD)', scale='log')